## TP2 - Aprendizaje Automático

Francisco Javier Piqueras Martínez

Realizamos los imports:

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow import keras
import matplotlib as mpl
import pandas as pd
import os
import matplotlib.pyplot as plt

Comprobamos versiones:

In [ ]:
tf.__version__

In [ ]:
keras.__version__

Cargamos el dataset de MINST de keras.

In [ ]:
MNIST_dataset = keras.datasets.mnist.load_data()

Dividimos en nuestro train set y test set

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = MNIST_dataset

In [ ]:
X_train_full.shape

In [ ]:
X_train_full.dtype

### Primero separamos el validation set del training set y escalamos cada pixel al rango 0-1.

Puesto que cada pixel toma un valor entre 0 y 255, vamos a dividir por 255. Dividimos por 255 como float (con el '.') para que el resultado. también tenga decimales.

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

### Tenemos 60000 imágenes en matrices de 28x28.

In [ ]:
plt.imshow(X_train[5], cmap="binary")
plt.axis('off')
plt.show()

### Creación del modelo

Vamos a intentar resolver el problema con una única MLP de 4 capas ocultas usando la Sequential API.
Como se trata de un problema de Multiclass classification con matrices de 28x28 usaremos:
- 28x28 = 784 input neurons
- 4 capas opcultas
- Neuronas por capa: 300
- 10 output neurons (una por clase)
- Función de activación de las capas ocultas: ReLU
- Función de activación de la output layer: Softmax
- Loss Function: Cross Entropy


In [ ]:
hidden_neurons = 300
class_num = 10

model1 = keras.models.Sequential()
model1.add(keras.layers.Flatten(input_shape=[28, 28]))
model1.add(keras.layers.Dense(hidden_neurons, activation=keras.activations.relu))
model1.add(keras.layers.Dense(hidden_neurons, activation=keras.activations.relu))
model1.add(keras.layers.Dense(hidden_neurons, activation=keras.activations.relu))
model1.add(keras.layers.Dense(hidden_neurons, activation=keras.activations.relu))
model1.add(keras.layers.Dense(class_num, activation=keras.activations.softmax))

In [ ]:
model1.summary()

### Compilamos el modelo

In [ ]:
model1.compile(loss="sparse_categorical_crossentropy", 
              optimizer="sgd", 
              metrics=["accuracy"])

### Entrenamos el modelo

Vamos a implementar Early Stopping y guardar el modelo con mejor rendimiento. También vamos a usar el callback de TensorBoard que propociona Keras para visualizar la curva de aprendizaje.

Creamos un método que genere carpetas parametrizado por la fecha. Esto lo usará TB para giardar los logs de cada run en una carpeta distinta.

In [ ]:
root_logdir = os.path.join(os.curdir, "tb_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

> Para solucionar un problema en OSX: 
> Error: Initializing libiomp5.dylib, but found libiomp5.dylib already initialized.

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

TB callback

In [ ]:
tb_cb = keras.callbacks.TensorBoard(get_run_logdir())

### Checkpoint callback para guardar el estado de los modelos en cada epoch.

Guardamos sólo el mejor para ahorrar espacio.

In [ ]:
ck_cb = keras.callbacks.ModelCheckpoint("intento1_model.h5", save_best_only=True)

Early Stop callback. Lo parametrizamos con 15 epoch de paciencia.

In [ ]:
es_cb = keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True)

### Finalmente entrenamos el modelo.

In [ ]:
history = model1.fit( X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[tb_cb, ck_cb, es_cb])

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

Usamos la extensión de TB.

In [ ]:
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir=./tb_logs --port=6008

Obtenemos un accuracy > 98%. Sin embargo, vamos a hacer hyperparameter tunning.

## Hyperparameter Tunning

Vamos a hcer un wrapper para nuestro modelo y que así podamos usar el RandomzedSearchCV.

In [ ]:
#keras.backend.clear_session()
#np.random.seed(42)
#tf.random.set_seed(42)

In [ ]:
def build_model(n_hidden=1, n_neurons=300, learning_rate=3e-3, input_shape=[28, 28]):
    model = keras.models.Sequential()
    # Input Layer
    model.add(keras.layers.Flatten(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation=keras.activations.relu))
    # Output Layer
    model.add(keras.layers.Dense(10, activation=keras.activations.softmax))
    
    SGDoptimizer = keras.optimizers.SGD(lr=learning_rate)
    
    model.compile(
        loss=keras.losses.sparse_categorical_crossentropy, 
        optimizer=SGDoptimizer,
        metrics=[keras.metrics.sparse_categorical_accuracy]
    )
    
    return model

In [ ]:
keras_wrapper = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

# Los valores comentados son los que yo usaría, sin embargo, los he simplificado para reducir el tiempo de entrenamiento. Lo más importante es que se entienda la intención.
param_distribs = {
    "n_hidden": [1,2, 3, 4, 5, 6],
    # Probamos de 50 en 50.
    "n_neurons": np.arange(50, 650, 50),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_wrapper, param_distribs, n_iter=20, cv=3)


Bajamos el número de epochs de patience para que no se alargue el entrenamiento.

In [ ]:
es_cb_unpatient = keras.callbacks.EarlyStopping(patience=10)

In [ ]:
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[es_cb_unpatient])

In [ ]:
rnd_search_cv.best_params_
# {'learning_rate': 0.01665249551851483, 'n_hidden': 1, 'n_neurons': 200}

Re-entrenamos con los parámetros óptimos

In [ ]:
hidden_layers = 1
hidden_neurons = 200
LR = 0.01665249551851483

In [ ]:
model_opt = build_model(n_hidden=hidden_layers, 
                        n_neurons=hidden_neurons, 
                        learning_rate=LR, 
                        input_shape=[28, 28])

In [ ]:
model_opt.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=LR),
              metrics=[keras.metrics.sparse_categorical_accuracy])

In [ ]:
opt_ck_cb = keras.callbacks.ModelCheckpoint("opt_model.h5", save_best_only=True)

In [ ]:
history_opt = model_opt.fit( X_train, y_train, epochs=150, validation_data=(X_valid, y_valid), callbacks=[tb_cb, opt_ck_cb, es_cb])

Volvemos a ver las gráficas de TB.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./tb_logs --port=6008

## Evaluamos los modelos

Modelo con 4 capas:

In [ ]:
model_1_eval = model1.evaluate(X_test, y_test)

In [ ]:
# [0.07861688811050262, 0.9761]
print(model_1_eval)

Modelo optimizado:

In [ ]:
model_opt_eval = model_opt.evaluate(X_test, y_test)

In [ ]:
# [0.06433085899604485, 0.9808]
print(model_opt_eval)